In [11]:
from sagemaker.pytorch import PyTorchModel
import sagemaker
import boto3
import pandas as pd
import numpy as np
import torch
import json

# the bucket that contains the compressed model artifact (model.tar.gz)
bucket_name = 'sept-16-test-bucket'
role = sagemaker.get_execution_role()

print('bucket name: {}'.format(bucket_name))
print('execution role: {}'.format(role))

bucket name: sept-16-test-bucket
execution role: arn:aws:iam::368560229227:role/service-role/AmazonSageMaker-ExecutionRole-20200916T213040


In [10]:
path_to_model = 's3://{}/model.tar.gz'.format(bucket_name)

# have to specify framework_version, otherwise will throw an error
pytorch_model = PyTorchModel(model_data=path_to_model, role=role, entry_point='inference.py', framework_version='1.3.1')
pytorch_model

In [11]:
# takes about 15 minutes to finish, '---' is the progress bar, '!' indicates that it's finished
predictor = pytorch_model.deploy(instance_type='ml.t2.medium', initial_instance_count=1)

---------------!

In [12]:
predictor

In [13]:
# the following code cell is used to load input from test set to see 
# if the prediction made by the trained model is around the correct value 

data_key_cam1 = '2d_points_0_to_1_cam1_test_with_shift.csv'
data_key_cam2 = '2d_points_0_to_1_cam2_test_with_shift.csv'

# path to the test dataframes in s3 bucket
data_location_cam1 = 's3://{}/{}'.format(bucket, data_key_cam1)
data_location_cam2 = 's3://{}/{}'.format(bucket, data_key_cam2)

df_cam1 = pd.read_csv(data_location_cam1)
df_cam2 = pd.read_csv(data_location_cam2)

cols = ['Sphere_X', 'Sphere_Y', 'Cube_X', 'Cube_Y']
cam1_sphere_cube_points = np.stack([df_cam1[col].values for col in cols], 1)
cam1_sphere_cube_points = torch.FloatTensor(cam1_sphere_cube_points)
cam2_sphere_cube_points = np.stack([df_cam2[col].values for col in cols], 1)
cam2_sphere_cube_points = torch.FloatTensor(cam2_sphere_cube_points)

# randomly set the time offset to be 3, and I randomly chose 67 to be the start index of the clip from cam1
time_offset = 3
cam1_clip_start_index = 67

# load the frames of cam1 starting at the 67th frame, ends at the 76th frame
clip_cam1 = cam1_sphere_cube_points[cam1_clip_start_index: (cam1_clip_start_index + 10)]
# load the frames of cam2 starting at the 70th frame, ends at the 79th frame
clip_cam2 = cam2_sphere_cube_points[(cam1_clip_start_index + time_offset) : (cam1_clip_start_index + 10 + time_offset)]

# convert the input date to a json object to be passed to the SageMaker Endpoint
input_json = json.dumps({"clip_cam1": clip_cam1.numpy().tolist(), "clip_cam2": clip_cam2.numpy().tolist()})

In [16]:
client = boto3.client('runtime.sagemaker')
 
# the EndpointName here is the name of the endpoint you just created by executing 
# the cell above that contains "pytorch_model.deploy", you can find the name under the "Instance --> Endpoint" section 
response = client.invoke_endpoint(EndpointName='pytorch-inference-2020-09-17-04-51-42-265',
                                  Body=input_json)
response_body = response['Body'] 
# print the prediction made by the pretrained model (model.tar.gz), which is the same as the time_offset I chose above, 
# which means that the model predicted the correct time offset based on those 2d positions of objects ('Sphere_X', 'Sphere_Y', 'Cube_X', 'Cube_Y') of the 2 cameras
print(response_body.read())

{"predicted time offset": 3.0}


In [ ]:
# delete the endpoint after use to save cost!!!
predictor.delete_endpoint()